In [ ]:
!pip install pandas 
!pip install sentence_transformers
!pip install spacy

!python -m spacy download en_core_web_lg

In [8]:
import numpy as np
import pandas as pd
import spacy
import pickle
from sentence_transformers import SentenceTransformer
import transformers as ppb
from datetime import datetime as dt


# Functies


De text_to_sent_list functie vormt een meegegeven tekst om naar 2 variabelen.
* De eerste (sent_clean) bevat een lijst met elke zin uit te tekst.
* De tweede (sent_embedding) bevat een lijst met embeddings(vectoren), elke embedding stelt een zin voor in de tekst.

Extra info:
* De attribuut 'nlp' is een taal model die de tekst gaat opsplitsen in zinnen.
* De attribuut 'embedder' is het BERT model die van elke zin een embedding maakt. 


In [9]:
def text_to_sent_list(text, nlp, embedder, min_len=2):
    ''' Returns cleaned article sentences and BERT sentence embeddings'''
    
    #convert to list of sentences
    text = nlp(text)
    sents = list(text.sents)
    #remove short sentences by threshhold                                                                                                
    sents_clean = [sentence.text for sentence in sents if len(sentence)> min_len]
    #remove entries with empty list
    sents_clean = [sentence for sentence in sents_clean if len(sentence)!=0]
    #embed sentences (deafult uses BERT SentenceTransformer)
    sents_embedding= np.array(embedder.encode(sents_clean, convert_to_tensor=True))
    
    return sents_clean, sents_embedding

# Script

Als eerst wordt er hier wordt de data en verschillende modellen opgehaald.


In [10]:
output_file = '../data/df_processed.pickle'
df_data = pd.read_csv('../dataset/BBCnews.csv')
nlp = spacy.load('en_core_web_lg')
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

Als volgt wordt er een dataframe met de nodige kolommen aangemaakt.  
Hierin wordt er per tekst/samenvatting set uit de dataset het volgende bijgehouden:
* subject : onderwerp
* text_clean : een lijst met de zinnen van de tekst
* text_embedding : een lijst met per zin van de tekst de embedding van die zin
* sum_clean : een lijst met de zinnen van de samenvatting 
* sum_embedding : een lijst met per zin van de samenvatting de embedding van die zin
 

In [ ]:
df = pd.DataFrame(columns=['subject','text_clean','text_embedding','sum_clean','sum_embedding'])
t1 = dt.now()
# clean sentences en embeddings maken en in de dataframe plaatsen voor elke text-samenvatting set
for index, row in df_data.iterrows():
    print(index)
    text_clean, text_embedding = text_to_sent_list(row['Articles'],nlp,embedder)
    sum_clean, sum_embedding = text_to_sent_list(row['Summaries'],nlp,embedder)

    new_row = {'subject':row['Subject'],'text_clean':text_clean, 'text_embedding':text_embedding,'sum_clean':sum_clean, 'sum_embedding':sum_embedding}

    df = df.append(new_row, ignore_index=True)


Als laatste wordt deze data opgeslagen in een pickle bestand.  
De data en de runtime wordt ook nog getoond.

In [ ]:
# save to pickle
with open(output_file, 'wb') as handle:                                     
    pickle.dump(df, handle)

print(df)
t2=dt.now()
print(t2-t1)